In [31]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.decomposition import PCA 
import featuretools as ft
# Regression Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Classification metrics
from sklearn.metrics import roc_curve, roc_auc_score
# Keras
import tensorflow as tf
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, MaxPooling2D, Activation
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
# Ignore ConvergenceWarning messages
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

In [44]:
# smaller datasets for automated features
ft        = pd.read_csv('train_ft.csv')
train     = pd.read_csv('application_train.csv')
bureau    = pd.read_csv('bureau.csv')
bureaubal = pd.read_csv('bureau_balance.csv')
prev      = pd.read_csv('previous_application.csv')
ccb       = pd.read_csv('credit_card_balance.csv')
insta     = pd.read_csv('installments_payments.csv')
pc        = pd.read_csv('POS_CASH_balance.csv')

In [46]:
ids = ft['SK_ID_CURR'].values
train = train.loc[train['SK_ID_CURR'].isin(ids)]
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
29,100034,0,Revolving loans,M,N,Y,0,90000.0,180000.0,9000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
35,100041,0,Cash loans,F,N,N,0,112500.0,450000.0,44509.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
38,100045,0,Cash loans,F,N,Y,0,99000.0,247275.0,17338.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
50,100058,0,Revolving loans,F,N,Y,0,54000.0,135000.0,6750.0,...,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307463,456203,0,Cash loans,F,N,Y,0,180000.0,247275.0,17338.5,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0
307469,456209,0,Cash loans,F,N,Y,0,202500.0,703728.0,29943.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
307489,456233,1,Cash loans,F,N,Y,0,225000.0,521280.0,23089.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
307495,456239,0,Cash loans,M,Y,N,0,180000.0,808650.0,23773.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
ids = train['SK_ID_CURR'].values
bureau = bureau.loc[bureau['SK_ID_CURR'].isin(ids)]
idsb = bureau['SK_ID_BUREAU'].values
bureaubal = bureaubal.loc[bureaubal['SK_ID_BUREAU'].isin(idsb)]
prev = prev.loc[prev['SK_ID_CURR'].isin(ids)]
ccb  = ccb.loc[ccb['SK_ID_CURR'].isin(ids)]
insta = insta.loc[insta['SK_ID_CURR'].isin(ids)]
pc   = pc.loc[pc['SK_ID_CURR'].isin(ids)]
print('bureau shape: ', bureau.shape)
print('bureaubal shape: ', bureaubal.shape)
print('prev shape: ', prev.shape)
print('ccb shape: ', ccb.shape)
print('insta shape: ', insta.shape)
print('pc shape: ', pc.shape)

bureau shape:  (91298, 17)
bureaubal shape:  (985077, 3)
prev shape:  (139110, 37)
ccb shape:  (90374, 23)
insta shape:  (1136337, 8)
pc shape:  (86056, 8)


In [48]:
tables = [train, bureau, bureaubal,prev,ccb, insta, pc ]
for features in tables:
    cols = features.select_dtypes(include = object).columns
    features[cols] = features[cols].fillna('Missing')
    cols = features.select_dtypes(exclude = object).columns
    features[cols] = features[cols].fillna(0)
    print(features.isnull().sum().sum())

0
0
0
0
0
0
0


In [49]:
train = pd.get_dummies(train, drop_first=True)
bureau = pd.get_dummies(bureau, drop_first= True)
prev = pd.get_dummies(prev, drop_first= True)

In [50]:
# a = group.get_group(lst[2])
# if a.shape[0] >= 5 :
#     a = a[:5]
# else:
#     m99 = np.ones((5-a.shape[0],a.shape[1]))*-99
#     m99 = pd.DataFrame(m99, columns=a.columns)
#     a = a.append(m99)
# a

In [51]:
idl = train['SK_ID_CURR'].values
bureau = bureau.sort_values('DAYS_CREDIT', ascending= False)
lst = bureau['SK_ID_CURR'].values
lst = list(set(lst))
lst.sort()
group = bureau.groupby('SK_ID_CURR')
b = []
j=0
for sk in idl:
    if sk in lst:
        a = group.get_group(lst[j])
        if a.shape[0] >= 5:
            a = a[:5]
        else:
            m99 = np.ones((5-a.shape[0],a.shape[1]))*-99
            m99 = pd.DataFrame(m99, columns=a.columns)
            a   = a.append(m99)
        a = a.drop(['SK_ID_CURR', 'SK_ID_BUREAU'], axis=1)    
        a = a.values.flatten().tolist()
        b.extend(a)
        j += 1
    else:
        m99 = np.ones((5,bureau.shape[1]))*-99
        m99 = pd.DataFrame(m99, columns=bureau.columns)
        m99 = m99.drop(['SK_ID_CURR', 'SK_ID_BUREAU'], axis=1)    
        m99 = m99.values.flatten().tolist()
        b.extend(m99)
b = np.array(b)
b = np.reshape(b,(train.shape[0], 5, bureau.shape[1]-2, 1))

In [52]:
# prev = prev.sort_values('DAYS_DECISION', ascending= False)
# lst = prev['SK_ID_CURR'].values
# lst = list(set(lst))
# lst.sort()
# group = prev.groupby('SK_ID_CURR')
# p = []
# j = 0
# for sk in idl:
#     if sk in lst:
#         a = group.get_group(lst[j])
#         if a.shape[0] >= 5:
#             a = a[:5]
#         else:
#             m99 = np.ones((5-a.shape[0],a.shape[1]))*-99
#             m99 = pd.DataFrame(m99, columns=a.columns)
#             a   = a.append(m99)
#         a = a.drop(['SK_ID_PREV', 'SK_ID_CURR'], axis=1)    
#         a = a.values.flatten().tolist()
#         p.extend(a)
#         j += 1
#     else:
#         m99 = np.ones((5,prev.shape[1]))*-99
#         m99 = pd.DataFrame(m99, columns=prev.columns)
#         m99 = m99.drop(['SK_ID_PREV', 'SK_ID_CURR'], axis=1)    
#         m99 = m99.values.flatten().tolist()
#         p.extend(m99)
# p = np.array(p)
# p = np.reshape(p,(train.shape[0], 5, prev.shape[1]-2, 1))

In [53]:
print('shape of channel(bureau):', b.shape)
# print('shape of channel(previous_app):', p.shape)

shape of channel(bureau): (30000, 5, 28, 1)


In [65]:
y = train['TARGET']
y = to_categorical(y,2)

In [66]:
batch_size = 256
epochs = 100
np.random.seed(5)        

model = Sequential()

#1st conv layer
model.add(Conv2D(filters = 32, kernel_size= (6,150), padding="same",
                 input_shape=(b.shape[1],b.shape[2],1),data_format="channels_last",
                  activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#2nd conv layer
model.add(Conv2D(32, (6,150), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

#FC1
model.add(Dense(units= 128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#FC2
model.add(Dense(units= 100, name= 'feature_extract', activation='relu'))
model.add(BatchNormalization())

#output FC
model.add(Dense(units= 2, activation='sigmoid'))
model.build()
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['AUC'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 5, 28, 32)         28832     
_________________________________________________________________
batch_normalization_12 (Batc (None, 5, 28, 32)         128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 14, 32)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 14, 32)         921632    
_________________________________________________________________
batch_normalization_13 (Batc (None, 2, 14, 32)         128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 224)              

In [67]:
early_stops = EarlyStopping(patience=5, monitor='val_AUC')
mc = ModelCheckpoint('best_model.h5',
                      monitor='val_loss', 
                      verbose=0, 
                      save_best_only=True)
model.fit(b, y, validation_split=0.05, callbacks=[early_stops, mc], batch_size= batch_size, epochs= epochs, verbose=1)

Train on 28500 samples, validate on 1500 samples
Epoch 1/100
28500/28500 [==============================] - 152s 5ms/sample - loss: 0.5411 - AUC: 0.8388 - val_loss: 0.3226 - val_AUC: 0.9426
Epoch 2/100
28500/28500 [==============================] - 153s 5ms/sample - loss: 0.2827 - AUC: 0.9290 - val_loss: 0.2452 - val_AUC: 0.9404
Epoch 3/100
28500/28500 [==============================] - 153s 5ms/sample - loss: 0.2611 - AUC: 0.9322 - val_loss: 0.2537 - val_AUC: 0.9343
Epoch 4/100
28500/28500 [==============================] - 157s 6ms/sample - loss: 0.2612 - AUC: 0.9326 - val_loss: 0.2389 - val_AUC: 0.9423
Epoch 5/100
28500/28500 [==============================] - 154s 5ms/sample - loss: 0.2587 - AUC: 0.9332 - val_loss: 0.2404 - val_AUC: 0.9414
Epoch 6/100
28500/28500 [==============================] - 154s 5ms/sample - loss: 0.2575 - AUC: 0.9359 - val_loss: 0.2409 - val_AUC: 0.9394
Epoch 7/100
28500/28500 [==============================] - 155s 5ms/sample - loss: 0.2572 - AUC: 0.9358 -

In [68]:
model.save('keras_model_conv2D.h5')
model = load_model('keras_model_conv2D.h5')
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('feature_extract').output)
intermediate_layer_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6_input (InputLayer)  [(None, 5, 28, 1)]        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 5, 28, 32)         28832     
_________________________________________________________________
batch_normalization_12 (Batc (None, 5, 28, 32)         128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 14, 32)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 14, 32)         921632    
_________________________________________________________________
batch_normalization_13 (Batc (None, 2, 14, 32)         128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1, 7, 32)          0   

In [69]:
#predict to get featured data
feauture_engg_data = intermediate_layer_model.predict(b)
feauture_engg_data = pd.DataFrame(feauture_engg_data)
print('feauture_engg_data shape:', feauture_engg_data.shape)

feauture_engg_data shape: (30000, 100)


In [70]:
# Renaming columns
new_col = []
for i in range(100):
    new_col.append('bfeat_%d'%(i+1))
feauture_engg_data.columns = new_col
feauture_engg_data.to_csv('100_feature.csv', index = False)
feauture_engg_data.head(5)  #The features are unnamed now

,bfeat_1,bfeat_2,bfeat_3,bfeat_4,bfeat_5,bfeat_6,bfeat_7,bfeat_8,bfeat_9,bfeat_10,...,bfeat_91,bfeat_92,bfeat_93,bfeat_94,bfeat_95,bfeat_96,bfeat_97,bfeat_98,bfeat_99,bfeat_100
0,0.286733,0.000000,0.00000,0.00000,1.556518,0.000000,0.257738,0.927,0.602099,1.230798,...,1.805967,0.921971,1.314762,0.000000,0.000000,0.893562,0.402721,0.509323,0.000000,2.026018
1,0.000000,1.718266,0.00000,2.00098,0.000000,1.281953,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.054308,0.000000,0.000000,0.000000,1.858672,0.000000
2,0.000000,1.718266,0.00000,2.00098,0.000000,1.281953,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.054308,0.000000,0.000000,0.000000,1.858672,0.000000
3,0.000000,1.718266,0.00000,2.00098,0.000000,1.281953,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.054308,0.000000,0.000000,0.000000,1.858672,0.000000
4,0.000000,0.000000,2.50021,0.00000,0.203495,0.000000,0.000000,0.000,1.316268,0.000000,...,0.154884,1.751562,3.570715,2.487453,0.000000,0.502090,2.424979,0.000000,0.000000,2.706565
